# Natural Language Processing

In [1]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


In [2]:
# Importing helper functions
from DanielBourke_HelperFunctions import create_tensorboard_callback, plot_loss_curves, compare_historys

### Analysing text dataset

In [3]:
# Loading data
import pandas as pd

train_df = pd.read_csv("NLP_text/train.csv")
test_df = pd.read_csv("NLP_text/test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Shuffling training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# Checking test dataframe
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# Checking number of training records
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
# Checking total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [8]:
# Visualising random samples
import random

random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[["text", "target"]][random_index : random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}")
    print("---")

Target: 1 (real disaster)
Text:
Rolling sandunes the gentle lapping of the sea the call of gulls and a nuclear reactor #sizewell http://t.co/X9CUiHIb5n
---
Target: 1 (real disaster)
Text:
#WorldWatchesFerguson #Florida @GovJayNixon @clairecmc How dare you turn our streets into a war zone -a war against CITIZENS?
---
Target: 0 (not real disaster)
Text:
@KristinDavis @UN @Refugees Thank you @UN and @Refugees for helping so many people in need all over the world.... https://t.co/yPvJgzqqqB Û_
---
Target: 1 (real disaster)
Text:
abcnews - Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern... http://t.co/mg5eAJElul
---
Target: 0 (not real disaster)
Text:
Keep thinking about it until I stepped on a broken glass pun tak sedar and I don't feel the pain also it's bleeding. Shit
---


### Creating validation data

In [9]:
# Splitting dataset
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1, # Allocating 10% to validation data
    random_state=42
)

In [10]:
# Checking dataset length
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [11]:
# Checking the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### Converting text to numbers

In [12]:
# Using text vectorisation
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False
)

In [13]:
# Finding the average number of tokens
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [14]:
# Setting up text vectorisation variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [15]:
# Fitting text vectorisation to the training dataset
text_vectorizer.adapt(train_sentences)

In [16]:
# Create a sample sentnence and tokenise it
sample_sentence = "There is a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [17]:
# Tokenising random sentence from the training set
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\ntokenised version:\n{text_vectorizer([random_sentence])}"
)

Original text:
News Wrap: UN warns female and child casualties are on the rise in Afghanistan http://t.co/vSvY1qe69t #pbs #iraq,
tokenised version:
[[  58 6666 1348 2234 2115    7  785  518   22   11    2  727    4 1439
     1]]


In [18]:
# Getting unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating Embedding layer

In [19]:
# Defining the layer
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length
)
embedding

In [20]:
# Get a random sentence
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\nembedded version:"
)

# Embed random sentence
embed_sentence = embedding(text_vectorizer([random_sentence]))
embed_sentence

Original text:
CIVIL WAR GENERAL BATTLE BULL RUN HERO COLONEL 2nd NEW HAMPSHIRE LETTER SIGNED ! http://t.co/Ot0tFFpBYB http://t.co/zaRBwep9LD,
embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03910953,  0.04944987, -0.00075934, ..., -0.03403198,
         -0.00348186, -0.01742106],
        [ 0.0041035 , -0.00746389,  0.01504979, ...,  0.0236407 ,
          0.02998877,  0.01439286],
        [ 0.01736666,  0.02454226,  0.01020164, ...,  0.00234209,
          0.00887488,  0.03583899],
        ...,
        [-0.04179144,  0.02842022,  0.04328972, ..., -0.043797  ,
         -0.00254587, -0.04064964],
        [-0.03600398,  0.01793792, -0.01776426, ..., -0.02122303,
          0.02432457,  0.04955358],
        [-0.03600398,  0.01793792, -0.01776426, ..., -0.02122303,
          0.02432457,  0.04955358]]], dtype=float32)>

In [21]:
# Checking single token's embedding
embed_sentence[0][0], embed_sentence[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03910953,  0.04944987, -0.00075934, -0.00431272, -0.03259523,
        -0.03593317,  0.01212875, -0.04515864, -0.01814132, -0.0021026 ,
        -0.01968093, -0.04616164,  0.0139874 , -0.03467422, -0.00062526,
        -0.03975097, -0.01643282, -0.02727976,  0.03836131,  0.04831917,
        -0.03405396, -0.04437586,  0.02378335, -0.04233317, -0.0256915 ,
        -0.00028577, -0.01532711,  0.01004393,  0.00557385, -0.00929018,
        -0.01044401,  0.04870481, -0.04779059,  0.02947042, -0.01351601,
        -0.04520537, -0.01884643,  0.01876217,  0.01113417,  0.03018364,
        -0.01309311, -0.01971263,  0.03411776, -0.03056052,  0.01944895,
         0.02422663, -0.03748114,  0.04300738, -0.0125602 ,  0.00091909,
         0.03636278, -0.0338579 ,  0.03670966, -0.03747848, -0.02095756,
         0.01968366, -0.01550616,  0.02724976, -0.01771086,  0.01087459,
        -0.01535462,  0.03529755, -0.03348463, -0.04700742,  0.02658183,
  

### Building base model

In [22]:
# Using SKLearn to build base model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenisation and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers
    ("clf", MultinomialNB()) # Model the text
])

# Fit the pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
# Evaluating baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Achieved accuracy: {baseline_score * 100:.2f}%")

Achieved accuracy: 79.27%


In [24]:
# Making predictions
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Function to evaluate model performance

In [25]:
# Importing SKLearn functions
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Developing function to evaluate accuracy, precision, recall and F1 scor
def calculate_results(y_true, y_pred):
    """
    Evaluate binary classification model
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and F1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }
    return model_results

In [26]:
# Getting baseline results
baseline_results = calculate_results(
    y_true=val_labels,
    y_pred=baseline_predictions
)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: a simple dense model

In [27]:
# Creating tensorboard callback
from DanielBourke_HelperFunctions import create_tensorboard_callback

# Creating log directory
SAVE_DIR = "model_logs"

In [28]:
# Building model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs) # Turn the input text into numbers
x = embedding(x) # Create embedding of numberised inputs
x = layers.GlobalAveragePooling1D()(x) # Condence the feature vector
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [29]:
# Getting model summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [30]:
# Compiling model
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [31]:
# Fitting the model
history_1 = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_1_dense"
    )]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20230115-140355
Epoch 1/5
215/215 [==============================] - 4s 15ms/step - loss: 0.6130 - accuracy: 0.6961 - val_loss: 0.5354 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4429 - accuracy: 0.8171 - val_loss: 0.4738 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3474 - accuracy: 0.8608 - val_loss: 0.4595 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2849 - accuracy: 0.8882 - val_loss: 0.4596 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 4s 16ms/step - loss: 0.2386 - accuracy: 0.9140 - val_loss: 0.4764 - val_accuracy: 0.7848


In [32]:
# Checking the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4764 - accuracy: 0.7848


[0.47637301683425903, 0.7847769260406494]

In [33]:
# Making predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [34]:
# First 10 predictions
model_1_pred_probs[:10]

array([[0.44770905],
       [0.7105849 ],
       [0.9981582 ],
       [0.14486265],
       [0.11354049],
       [0.947634  ],
       [0.88502544],
       [0.9941534 ],
       [0.97123855],
       [0.29792026]], dtype=float32)

In [35]:
# Converting model predictions to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
# Calculating model 1 results
model_1_results = calculate_results(
    y_true=val_labels,
    y_pred=model_1_preds
)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7867627887573655,
 'recall': 0.7847769028871391,
 'f1': 0.7828250820874111}

In [37]:
# Retrieving baseline results
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [38]:
# Comparing model results
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualising learned embeddings

In [39]:
# Getting vocabulary from the text vectorisation layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [40]:
# Checking model 1 details
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
# Getting the weights
embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights, embed_weights[0].shape

([array([[ 0.06417161,  0.01519952, -0.06827231, ..., -0.00054693,
          -0.02690474, -0.00448329],
         [-0.03435826,  0.01822414, -0.02051552, ..., -0.02347109,
           0.02209371,  0.04865316],
         [ 0.04357785,  0.01337285, -0.02630139, ...,  0.06265885,
           0.02366419, -0.05968196],
         ...,
         [ 0.01998829, -0.00305961,  0.00864701, ..., -0.00789784,
          -0.01902278, -0.04017846],
         [ 0.07212634,  0.01228446, -0.02136814, ...,  0.04640999,
           0.02045361,  0.01623856],
         [ 0.02796545,  0.10167719, -0.0370875 , ...,  0.03066149,
           0.02718702, -0.08076494]], dtype=float32)],
 (10000, 128))

In [42]:
# Creating embedding files (sample from TF Word Embedding documentation)
import io

out_v = io.open("vectors.tsv", "w", encoding="utf-8")
out_m = io.open("metadata.tsv", "w", encoding="utf-8")

for index, word in enumerate(words_in_vocab):
    if index == 0:
        continue # Skip 0, as it's a padding
    vec = embed_weights[0][index]
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

Generated vectors.tsv and metadata.tsv can be uploaded into Tensorflow Embedding Projector tool at https://projector.tensorflow.org/ to visualise weights in 3D space.

### Model 2 - RNN (Recurrent Neural Network) with LSTM (Long Short-Term Memory)

In [43]:
# CReating an RNN model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) # Sequences required when you stack LSTM layers
# print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [44]:
# Checking model's summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [45]:
# Compiling the model
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [46]:
# Fittting the model
history_2 = model_2.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_2_LSTM"
    )]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230115-140414
Epoch 1/5
215/215 [==============================] - 10s 29ms/step - loss: 0.2168 - accuracy: 0.9204 - val_loss: 0.5235 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1558 - accuracy: 0.9418 - val_loss: 0.6328 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1272 - accuracy: 0.9523 - val_loss: 0.6671 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 5s 26ms/step - loss: 0.1010 - accuracy: 0.9606 - val_loss: 0.7766 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0831 - accuracy: 0.9672 - val_loss: 0.9561 - val_accuracy: 0.7756


In [47]:
# Meking predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[5.0655664e-03],
       [7.9005218e-01],
       [9.9976742e-01],
       [2.2368792e-02],
       [2.8219089e-04],
       [9.9891210e-01],
       [6.4708853e-01],
       [9.9987787e-01],
       [9.9974960e-01],
       [3.0192614e-01]], dtype=float32)

In [48]:
# Converting pred probs to label format
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
#Calculate model 2 results
model_2_results = calculate_results(
    y_true=val_labels,
    y_pred=model_2_preds
)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7802047087708873,
 'recall': 0.7755905511811023,
 'f1': 0.7723569237171506}

### Model 3 - RNN with GRU (gated recurrent unit)

In [50]:
# Building the model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x) # Sequences, if GRU are stuck on each other
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x) # If we want to consolidate GRU layer with sequences
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [51]:
# Getting model summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [52]:
# Compiling the model
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [53]:
# Fitting the model
history_3 = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_3_GRU"
    )]
)

Saving TensorBoard log files to: model_logs/model_3_GRU/20230115-140448
Epoch 1/5
215/215 [==============================] - 9s 27ms/step - loss: 0.1524 - accuracy: 0.9391 - val_loss: 0.7385 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0839 - accuracy: 0.9688 - val_loss: 0.9156 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0729 - accuracy: 0.9724 - val_loss: 0.8051 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0582 - accuracy: 0.9762 - val_loss: 0.9416 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0502 - accuracy: 0.9787 - val_loss: 1.1723 - val_accuracy: 0.7808


In [54]:
# Meking predictions
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 1s 7ms/step


array([[3.04508489e-04],
       [8.72708082e-01],
       [9.99888122e-01],
       [1.35886058e-01],
       [6.78687502e-05],
       [9.99800086e-01],
       [9.63634968e-01],
       [9.99961257e-01],
       [9.99915898e-01],
       [9.07369256e-01],
       [4.41405660e-04],
       [9.64353502e-01],
       [2.16167988e-04],
       [2.30987623e-01],
       [1.32624322e-04],
       [9.43428662e-04],
       [4.87239042e-04],
       [2.85887305e-04],
       [1.76380444e-02],
       [9.99717772e-01],
       [9.99981284e-01],
       [5.44067079e-05],
       [9.99554098e-01],
       [1.21992640e-03],
       [9.99890745e-01],
       [9.99933243e-01],
       [8.98943923e-04],
       [2.49236095e-04],
       [1.35301292e-04],
       [3.88348907e-01],
       [9.95594323e-01],
       [5.57709427e-04],
       [1.15396708e-01],
       [5.90767944e-03],
       [4.85753030e-01],
       [3.25351954e-01],
       [9.99820113e-01],
       [1.82585508e-01],
       [3.22926678e-02],
       [9.99943733e-01],


In [55]:
# Converting model 3 pred probs to label format
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [56]:
# Calculate model 3 results
model_3_results = calculate_results(
    y_true=val_labels,
    y_pred=model_3_preds
)
model_3_results

{'accuracy': 78.08398950131233,
 'precision': 0.7823342077218286,
 'recall': 0.7808398950131233,
 'f1': 0.7790338643605079}

### Model 4 - Bidirectional RNN

In [57]:
# Create a model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidir")

In [58]:
# Checking model details
model_4.summary()

Model: "model_4_bidir"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [59]:
# Compiling the model
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [60]:
# Fitting the model
history_4 = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_4_bidir"
    )]
)

Saving TensorBoard log files to: model_logs/model_4_bidir/20230115-140520
Epoch 1/5
215/215 [==============================] - 16s 36ms/step - loss: 0.1061 - accuracy: 0.9693 - val_loss: 0.8600 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0516 - accuracy: 0.9761 - val_loss: 1.1719 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0464 - accuracy: 0.9794 - val_loss: 1.2445 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0418 - accuracy: 0.9800 - val_loss: 1.4125 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0371 - accuracy: 0.9801 - val_loss: 1.3751 - val_accuracy: 0.7690


In [61]:
# Making predictions
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 11ms/step


array([[1.4174721e-01],
       [7.5514436e-01],
       [9.9996418e-01],
       [9.8600216e-02],
       [1.1236031e-05],
       [9.9976200e-01],
       [9.4622087e-01],
       [9.9999058e-01],
       [9.9997622e-01],
       [8.7807655e-01]], dtype=float32)

In [62]:
# Adjusting model's probs to the label format
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [63]:
# Checking results of predictions
model_4_results = calculate_results(
    y_true=val_labels,
    y_pred=model_4_preds
)
model_4_results

{'accuracy': 76.9028871391076,
 'precision': 0.7693974434525301,
 'recall': 0.7690288713910761,
 'f1': 0.7676527036976099}

### Model 5 - Conv1D

In [64]:
# Testing embedding layer
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer("This is a test sentence"))
# print(embedding_test.shape)
embedding_test = tf.expand_dims(embedding_test, axis=0)
# print(embedding_test.shape)
conv_1d = layers.Conv1D(
    filters=32,
    kernel_size=5,
    activation="relu",
    padding="valid"
)
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

### Model 5 - 1D Convolutional Network layer

In [65]:
# Creating a model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(
    filters=64,
    kernel_size=5,
    strides=1,
    activation="relu",
    padding="valid"
)(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d")

In [66]:
# Checking model details
model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [67]:
# Compiling the model
model_5.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [68]:
# Fitting the model
history_5 = model_5.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_5_conv1d"
    )]
)

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230115-140601
Epoch 1/5
215/215 [==============================] - 6s 23ms/step - loss: 0.1282 - accuracy: 0.9555 - val_loss: 0.8809 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0754 - accuracy: 0.9720 - val_loss: 1.0246 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0615 - accuracy: 0.9762 - val_loss: 1.1349 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0558 - accuracy: 0.9772 - val_loss: 1.1829 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0505 - accuracy: 0.9788 - val_loss: 1.2500 - val_accuracy: 0.7677


In [69]:
# Making predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 5ms/step


array([[3.3906347e-01],
       [8.8784599e-01],
       [9.9995393e-01],
       [5.0555877e-02],
       [5.5122968e-08],
       [9.9883574e-01],
       [9.8082507e-01],
       [9.9998611e-01],
       [9.9999970e-01],
       [9.3652105e-01]], dtype=float32)

In [70]:
# Converting probs into label format
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [71]:
# Evaluating model's predictions
model_5_results = calculate_results(
    y_true=val_labels,
    y_pred=model_5_preds
)
model_5_results

{'accuracy': 76.77165354330708,
 'precision': 0.7681410880728078,
 'recall': 0.7677165354330708,
 'f1': 0.7662770891654436}

### Model 6 - TensorflowHub pretrained model

In [72]:
# Reusing USE (Universal Sentence Encoder) model
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([
    sample_sentence,
    "When you apple USE to text, it converts it into numbers"
])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01602832  0.01068848  0.02425467 -0.01405769  0.01434426  0.08292625
  0.01963372  0.06160142 -0.00352702 -0.01216412  0.00978647 -0.01248495
  0.01232345  0.09748451  0.06141113 -0.03728353  0.01860884 -0.04669856
  0.00413913 -0.06363907 -0.02469898  0.02713691  0.02284444 -0.00210026
 -0.00630592 -0.03964961  0.02220404  0.00115077 -0.03132177  0.00119527
 -0.0401255   0.04561892 -0.01530598 -0.00175918  0.02173131 -0.08450424
  0.03340026  0.04604554 -0.0248025  -0.08681665  0.00702694 -0.00770478
 -0.01434539  0.07814164 -0.10676058 -0.05152997 -0.00858155 -0.03232232
 -0.03871097  0.02581467], shape=(50,), dtype=float32)


In [73]:
# Sampling shape
embed_samples[0].shape

TensorShape([512])

In [74]:
# Creating layer with USE pretrained model
sentence_encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape=[],
    dtype=tf.string,
    trainable=False,
    name="USE"
)

In [75]:
# Building a new model with pretrained layer
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    # layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_use")

In [76]:
# Compiling the model
model_6.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [77]:
# Fitting the model
history_6 = model_6.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_6_use"
    )]
)

Saving TensorBoard log files to: model_logs/model_6_use/20230115-182143
Epoch 1/5
215/215 [==============================] - 11s 23ms/step - loss: 0.6476 - accuracy: 0.7399 - val_loss: 0.6137 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.5809 - accuracy: 0.7889 - val_loss: 0.5641 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.5384 - accuracy: 0.7962 - val_loss: 0.5328 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.5097 - accuracy: 0.7994 - val_loss: 0.5114 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4897 - accuracy: 0.8009 - val_loss: 0.4971 - val_accuracy: 0.7913


In [78]:
# Checking the model details
model_6.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [79]:
# Making predictions
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 11ms/step


array([[0.36288318],
       [0.6738508 ],
       [0.8496174 ],
       [0.37130713],
       [0.64633924],
       [0.7301284 ],
       [0.825036  ],
       [0.8416656 ],
       [0.75820655],
       [0.18755515]], dtype=float32)

In [80]:
# Converting prediction probs into label format
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [81]:
# Calculating model's performance metrics
model_6_results = calculate_results(
    y_true=val_labels,
    y_pred=model_6_preds
)
model_6_results

{'accuracy': 79.13385826771653,
 'precision': 0.7918101082010836,
 'recall': 0.7913385826771654,
 'f1': 0.7902401820506462}

### Model 7 - Pretrained USE with 10% of training data

In [82]:
# Creating subset of 10% training data
train_data_10p = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10p = train_data_10p["text"].to_list()
train_labels_10p = train_data_10p["target"].to_list()
len(train_sentences_10p), len(train_labels_10p)

(761, 761)

In [83]:
# Checking the length of original dataset
len(train_df_shuffled)

7613

In [84]:
# Checking the number of targets - 10%
train_data_10p["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

In [85]:
# Checking the number of targets - Full set
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [86]:
# Cloning model 6
model_7 = tf.keras.models.clone_model(model_6)

# Compiling the model
model_7.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

# Checking the model details
model_7.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [87]:
# Fitting the model
history_7 = model_7.fit(
    train_sentences_10p,
    train_labels_10p,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_7_use"
    )]
)

Saving TensorBoard log files to: model_logs/model_7_use/20230115-210344
Epoch 1/5
24/24 [==============================] - 7s 69ms/step - loss: 0.6809 - accuracy: 0.6715 - val_loss: 0.6737 - val_accuracy: 0.7152
Epoch 2/5
24/24 [==============================] - 1s 37ms/step - loss: 0.6685 - accuracy: 0.7385 - val_loss: 0.6622 - val_accuracy: 0.7533
Epoch 3/5
24/24 [==============================] - 1s 25ms/step - loss: 0.6572 - accuracy: 0.7530 - val_loss: 0.6512 - val_accuracy: 0.7651
Epoch 4/5
24/24 [==============================] - 1s 30ms/step - loss: 0.6466 - accuracy: 0.7661 - val_loss: 0.6406 - val_accuracy: 0.7730
Epoch 5/5
24/24 [==============================] - 1s 30ms/step - loss: 0.6364 - accuracy: 0.7740 - val_loss: 0.6308 - val_accuracy: 0.7795


In [88]:
# Making predictions
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 15ms/step


array([[0.48890504],
       [0.51846015],
       [0.5969771 ],
       [0.4629081 ],
       [0.5366968 ],
       [0.5619528 ],
       [0.5732625 ],
       [0.62088543],
       [0.5592856 ],
       [0.42308325]], dtype=float32)

In [89]:
# Converting pred probs to label format
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [91]:
# Evaluating model predictions
model_7_results = calculate_results(
    y_true=val_labels,
    y_pred=model_7_preds
)
model_7_results

{'accuracy': 77.95275590551181,
 'precision': 0.7798994748039346,
 'recall': 0.7795275590551181,
 'f1': 0.7796651050286317}